In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_directory = "../Data/Study/"
hololenses = ['A/', 'B/']
intermediate = ''
subdirectory = "recalibrated_calibration/"

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [4]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [5]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [8]:
def calculate_errors(output_subdirectory):
    for hololens in hololenses:
        participantsList = os.listdir(data_directory + hololens)
        for participant in participantsList:
            print(participant)
            if not participant == '.DS_Store':
                if os.path.exists(data_directory + hololens + participant + '/' + intermediate + output_subdirectory) and len(os.listdir(data_directory + hololens + participant + '/' + intermediate + output_subdirectory)) == 12:
                    print('already calculated')
                    continue
                fileList = os.listdir(data_directory + hololens + participant + '/' + intermediate + subdirectory)
                for file in fileList:
                    if '.csv' in file:
                        filePath = data_directory + hololens + participant + '/' + intermediate
                        df = pd.read_csv(filePath + subdirectory + file)
                        error_df = calculate_euclidean_error(df)
                        error_df = calculate_cosine_error(error_df)
                        if not os.path.exists(filePath + output_subdirectory):
                            os.makedirs(filePath + output_subdirectory)
                        error_df_output_path = filePath + output_subdirectory + '/' + file
                        error_df.to_csv(error_df_output_path, index=False)
                        print(file)
                        print("uncalibrated: " + str(error_df['euclideanError'].mean()))
                        print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
                        print("cosine: " + str(error_df['cosineError'].mean()))
                        print("")

In [9]:
calculate_errors("error/recalibrated_calibration")

03
already calculated
04
already calculated
32
already calculated
34
already calculated
05
already calculated
02
already calculated
.DS_Store
18
already calculated
27
already calculated
29
already calculated
10
already calculated
19
already calculated
43
wsBodyConstrained_20230518093547_shifted0.csv
uncalibrated: 28.22632754442796
calibrated: 28.885064037144303
cosine: 27.294367354735602

calibration_20230518092046_shifted0.csv
uncalibrated: 8.04688548628083
calibrated: 1.9062445358546531
cosine: 7.837273901511007

wsBodyConstrained_20230518092223_shifted0.csv
uncalibrated: 27.04800715106551
calibrated: 27.64038923382835
cosine: 26.17359078534333

calibration_20230518093425_shifted0.csv
uncalibrated: 4.6165858597349825
calibrated: 1.8414329939982086
cosine: 4.509688181865919

hallway_20230518092703_shifted0.csv
uncalibrated: 24.113978797184217
calibrated: 20.399591028842497
cosine: 23.500772385996516

ssHeadConstrained_20230518091535_shifted28.csv
uncalibrated: 6.4555378230966
calibrat